In [3]:
from tensorflow.python.client import device_lib
from tensorflow.python.keras.applications.inception_v3 import InceptionV3
from tensorflow.python.keras.applications.vgg16 import VGG16
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.optimizer_v2.adam import Adam
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.layers.core import Dense

In [4]:
image_generator = ImageDataGenerator(rescale=1.0/255, rotation_range=15, zoom_range=0.15, width_shift_range=0.05, height_shift_range=0.05)

In [5]:
train_generator = image_generator.flow_from_directory("../cats_and_dogs/train", target_size=(224, 224),
                                                          class_mode= 'categorical', batch_size=1)

Found 7999 images belonging to 2 classes.


In [7]:
test_generator = image_generator.flow_from_directory("../cats_and_dogs/test", target_size=(224, 224),
                                                         class_mode= 'categorical', batch_size=1)

Found 400 images belonging to 2 classes.


# Модель vgg16

In [8]:
vgg16 = VGG16(weights='imagenet', include_top=True)
vgg16.summary()

553467904/553467096 [==============================] - 57s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
___________________________________________________________

In [11]:
classification_layer = Dense(2, activation='softmax', name="images_prediction")
out = classification_layer(vgg16.layers[-2].output)
model = Model(vgg16.input, out)
model.summary()

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

In [12]:
for l, layer in enumerate(model.layers[:-1]):
    layer.trainable = False

model.layers[-1].trainable = True

In [13]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [15]:
model.fit_generator(train_generator, steps_per_epoch=2500, epochs=5, validation_data = test_generator,
          validation_steps = 200)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/5
2500/2500 [==============================] - 501s 200ms/step - loss: 0.6231 - accuracy: 0.7544 - val_loss: 0.4171 - val_accuracy: 0.8300
Epoch 2/5
2500/2500 [==============================] - 498s 199ms/step - loss: 0.4944 - accuracy: 0.8248 - val_loss: 0.8659 - val_accuracy: 0.7450
Epoch 3/5
2500/2500 [==============================] - 534s 213ms/step - loss: 0.6449 - accuracy: 0.8084 - val_loss: 0.4383 - val_accuracy: 0.8550
Epoch 4/5
2500/2500 [==============================] - 512s 205ms/step - loss: 0.5951 - accuracy: 0.8232 - val_loss: 0.4883 - val_accuracy: 0.8550
Epoch 5/5
2500/2500 [==============================] - 508s 203ms/step - loss: 0.6648 - accuracy: 0.8184 - val_loss: 0.5305 - val_accuracy: 0.8500


In [16]:
loss, accuracy = model.evaluate_generator(test_generator, steps=200)

print('Loss:', loss)
print('Accuracy:', accuracy)

Instructions for updating:
Please use Model.evaluate, which supports generators.
Loss: 0.5007706880569458
Accuracy: 0.8500000238418579
